In [ ]:
!pip install --upgrade gspread
!pip install mpld3
!pip install xlsxwriter

from google.colab import drive 
drive.mount('/content/gdrive')
from google.colab import auth
auth.authenticate_user()

import gspread
from google.auth import default
creds, _ = default()

gc = gspread.authorize(creds)
%matplotlib notebook
import mpld3
mpld3.enable_notebook()

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import scipy
import math

from scipy import signal
from scipy.signal import find_peaks,peak_prominences
from scipy.signal import butter,filtfilt
from scipy.fft import fft, fftfreq
from scipy.signal import cheby2
from scipy.signal import iirnotch
from scipy.signal import medfilt
from scipy.interpolate import interp1d

from numpy import array, sign, zeros


def get_FFT(data, fs): # fs: sampling frequency
    N = len(data) 
    # sample spacing
    T = 1.0 / fs
    yf = fft(data)
    xf = fftfreq(N, T)[:N//2] # positive frequencies of the spectrum
    yf_mags = 2.0/N * np.abs(yf[0:N//2]) # magnitude values of frequency components
    
    return xf, yf_mags

def adaptive_notch_filt(data, samp_freq, harmonic_freq):
    freq, amp = get_FFT(data, samp_freq)
    ind_range = np.where(np.logical_and(freq>= harmonic_freq - 2, freq <= harmonic_freq + 2))[0]
    
    max_loc = np.argmax(amp[ind_range])
    notch_freq = np.round(freq[ind_range][max_loc], 1)
    
    fn = 0.5*samp_freq
    fc = notch_freq
    Q = 3
    b, a = iirnotch(fc / fn, Q)
    filt_signal = filtfilt(b, a, data, axis = 0)
    return filt_signal

def adp_notch_filter(rec_data, fs):
    notch_filt1 = adaptive_notch_filt(rec_data, fs, 50)
    return notch_filt1

def butter_bandpass_filter(data,lowcutoff,highcutoff, fs, order):
    nyq = 0.5 * fs # Nyquist Frequency
    low_cutoff = lowcutoff / nyq
    high_cutoff=highcutoff/nyq
    # Get the filter coefficients 
    b, a = butter(order,[low_cutoff,high_cutoff], btype='bandpass', analog=False)
    y = filtfilt(b, a,data)
    return y

def get_fft_spec(sig, samp_freq, low, high): # get freq amps values in the specific range
    freq_magnitudes, freqs, __ = plt.magnitude_spectrum(sig, Fs = samp_freq)
    indices = np.where((freqs>low) & (freqs<=high))[0]
    req_freqs = freqs[indices]
    req_specs = freq_magnitudes[indices]
    
    return req_specs, req_freqs

def mov_avg(data, w_d):
    return np.convolve(data, np.ones(w_d)/w_d, 'same')

def findLocalMaxima(arr):
 
    # Empty lists to store points of
    # local maxima and minima
    mx = []
    max_values = []
  
    # Iterating over all points to check
    # local maxima and local minima
    n = len(arr)
    for i in range(1, n-1):
 
        # Condition for local maxima
        if(arr[i-1] < arr[i] > arr[i + 1]):
            mx.append(i)
            max_values.append(arr[i])
    
    return mx

def get_closeby_pnts(pnts1, pnts2, max_abs_range): # max_abs_range >=0 and less than the half of the pulse width
    n1 = pnts1
    n2 = pnts2
    closeby_pnts1 = []
    closeby_pnts2 = []
    
    for i in range(len(n1)):
        for j in range(len(n2)):
            if abs(n1[i]-n2[j])<=max_abs_range:
                closeby_pnts1.append(n1[i])
                closeby_pnts2.append(n2[j])
                
    return np.array(closeby_pnts1), np.array(closeby_pnts2)


def get_envelopes(sig, ipln_method):
    s = sig
    q_u = zeros(len(sig))
    q_l = zeros(len(sig))
    
    f_mags, freqs = get_fft_spec(sig, fs, 2-1.2, 2+0.9)  # get the frequency spectrum within the range 2-0.9:2+0.9

    # get first harmonic frequency
    f1_harmonic = freqs[np.argmax(f_mags)]
    
    PP_min = int(fs/f1_harmonic) - (fs/6)  # fs/6 has chosen based on working on data

    # find out the valleys in each of the wavelength
    y = sig
    y_inv = max(y) - y
    valleys,_ = find_peaks(y_inv, distance = PP_min)
    sig_valleys = sig[valleys]
    l_x = valleys; l_y = sig_valleys

    #Prepend the first value of (s) to the interpolating values. This forces the model to use the same starting point for both the upper and lower envelope models.

    u_x = []
    u_y = []

    l_x = [0] + list(l_x)
    l_y = [np.mean(sig_valleys[:6])] + list(l_y)

#     #Detect peaks and troughs and mark their location in u_x,u_y,l_x,l_y respectively.

    for k in range(1,len(s)-1):
        if (sign(s[k]-s[k-1])==1) and (sign(s[k]-s[k+1])==1):
            u_x.append(k)
            u_y.append(s[k])

#         if (sign(s[k]-s[k-1])==-1) and ((sign(s[k]-s[k+1]))==-1):
#             l_x.append(k)
#             l_y.append(s[k])
    
    #Append the last value of (s) to the interpolating values. This forces the model to use the same ending point for both the upper and lower envelope models.
    
    u_x.append(len(s)-1)
    u_y.append(s[-1])

    l_x.append(len(s)-1)
    l_y.append(np.mean(sig_valleys[-6:]))

    #Fit suitable models to the data. Here I am using cubic splines, similarly to the MATLAB example given in the question.

    u_p = interp1d(u_x,u_y, kind = ipln_method, bounds_error = False, fill_value=0.0)
    l_p = interp1d(l_x,l_y, kind = ipln_method, bounds_error = False, fill_value=0.0)

    #Evaluate each model over the domain of (s)
    for k in range(0,len(sig)):
        q_u[k] = u_p(k)
        q_l[k] = l_p(k)
        
    return q_u, q_l

def get_BW_removed_sig(sig, samp_freq, inpln_method):
    upper_epe, lower_epe = get_envelopes(sig, inpln_method)
#     dup_epe,dlow_epe = get_envelopes(lower_epe, "linear")
    # # tup_epe, tlow_epe = get_envelopes(dlow_epe)

    bw_removed_sig = sig - mov_avg(lower_epe, samp_freq)
    
    return upper_epe, lower_epe, bw_removed_sig


def get_valleys(sig1, sig2, samp_freq):
    
    fs = samp_freq  # sample rate, Hz

    signals = [sig1, sig2]

    signals_valleys = []
    
    for sig in signals:

        f_mags, freqs = get_fft_spec(sig, fs, 2-1.2, 2+0.9)  # get the frequency spectrum within the range 2-0.9:2+0.9

        # get first harmonic frequency
        f1_harmonic = freqs[np.argmax(f_mags)]

        # [f_min, f_max] = get_harmonics_width(bpass_sigB, fs, f1_harmonic)[0]
        # RR_min = int(fs / f_max); RR_max = int(fs / f_min);

        PP_min = int(fs/f1_harmonic) - (fs/6)  # fs/6 has chosen based on working on data

        # find out the valleys in each of the wavelength
        y = sig
        y_inv = max(y) - y
        valleys,_ = find_peaks(y_inv, distance = PP_min)
        
        signals_valleys.append(valleys)
        
    valid_valleys1, valid_valleys2  = get_closeby_pnts(signals_valleys[0], signals_valleys[1], int(fs/(2*f1_harmonic)))
    
    return valid_valleys1, valid_valleys2


def get_valid_first_peaks(vals, sig1, sig2):
    
    # get first peaks in each valley to valley 
    valid_first_peaks1 = []
    valid_first_peaks2 = []
    
    N = len(vals)
    
    for v_i in range(N-1):
        
        pulse_peaks1 = sorted(findLocalMaxima(sig1[vals[v_i]:vals[v_i + 1]]))
        pulse_peaks2 = sorted(findLocalMaxima(sig2[vals[v_i]:vals[v_i + 1]]))
        
#         first_peaks1, first_peaks2  = closeby_pnts(pulse_peaks1[0], pulse_peaks2[0], 60)
        
        if len(pulse_peaks1)!=0 and len(pulse_peaks2)!=0:
            valid_first_peaks1.append(pulse_peaks1[0] + vals[v_i])
            valid_first_peaks2.append(pulse_peaks2[0] + vals[v_i])
            
    return np.array(valid_first_peaks1), np.array(valid_first_peaks2)


def get_preprocessed_sig(sig, samp_freq, order, inpln_method):
    
    fs = samp_freq   

    lowcutoff = 1    
    highcutoff = 5  
    order = 3       

    notch_sig = adp_notch_filter(sig, fs)

    bpass_sig = butter_bandpass_filter(notch_sig, lowcutoff, highcutoff, fs, order)
    
    peak_envpe, val_envpe, processed_sig = get_BW_removed_sig(bpass_sig, fs, inpln_method)
    
    preprocessed_sig = processed_sig / (peak_envpe - val_envpe)
    
    return peak_envpe, val_envpe, preprocessed_sig

def drop_nan(x):
    return x[~np.isnan(x)]

# ******************************************  Store PTT values in excel ***************************************** #

print("enter the file path")
user_fp = input()
file_path = str(user_fp)

# file_path = r"E:\Ramu_Pittala\Multi Wavelength PTT Analysis\Good Signals PTT Analysis\48.2_110522_BP1_V_420_6_171.csv"

df=pd.read_csv(file_path,skiprows=36, header = None)

# print("enter excel file key")
# excel_fp = input()
# excel_file_key = excel_fp

# wb = gc.open_by_key(excel_file_key).sheet1
file_name = file_path.split("/")[-1][:-4]
sh = gc.create(file_name + "_ptt_sheet")
wb = sh.sheet1
# wb = sh.add_worksheet(title="ptt combination sheet", rows=100, cols=20)

fs = int(file_path.split("_")[-3])
num_measures = int(file_path.split("_")[-2])

if num_measures == 6:
    column_ind = [1, 5, 9, 13, 17, 21]
    mm_names = ['Meas1', 'Meas2', 'Meas3', 'Meas4', 'Meas5', 'Meas6']
    
elif num_measures == 4:
    column_ind = [1, 5, 9, 13]
    mm_names = ['Meas1', 'Meas2', 'Meas3', 'Meas4']
    
elif num_measures == 2:
    column_ind = [1, 5]
    mm_names = ['Meas1', 'Meas2']
    
ptt_names = ['PTT with First Peaks', 'PTT with Max Slopes', 'PTT with Valleys']
    
bw_length = len(column_ind) + 2

row_ptt_mean_first_peaks = []
row_ptt_mean_max_slopes = []
row_ptt_mean_valleys = []

for i in range(3):
    row = 2
    p = 2 + bw_length*i
    q = p
    wb.update_cell(row-1, p, ptt_names[i])
    c = 0
    d = row + 1
    for j in range(p, p+num_measures):
        wb.update_cell(row,j, mm_names[c])
        wb.update_cell(d, q-1, mm_names[c])
        c += 1
        d += 1

row = 3
col = 3

for r in range(len(column_ind)-1):
    
    col_ptt_mean_first_peaks = []
    col_ptt_mean_max_slopes = []
    col_ptt_mean_valleys = []
    
    wb.update_cell(row,col-1, 0)
    wb.update_cell(row,col + bw_length-1 ,0)
    wb.update_cell(row,col + (bw_length*2)-1, 0)
    
    for c in range(r+1, len(column_ind)):
        
        sig_1 = drop_nan(np.array(df.iloc[:, column_ind[r]])*(-1))
        sig_2 = drop_nan(np.array(df.iloc[:, column_ind[c]])*(-1))

        # ======================  Get different fiducial points ====================== ###

        # :::::::::::::::::  Get pre-processed signals :::::::::::::::::::::::::::::: #
        val_epe1, peak_epe1, proc_sig1 = get_preprocessed_sig(sig_1, fs, 3, "quadratic")
        val_epe2, peak_epe2, proc_sig2 = get_preprocessed_sig(sig_2, fs, 3, "quadratic")

        vals1, vals2 = get_valleys(proc_sig1, proc_sig2, fs)
        sig_peaks1, sig_peaks2 = get_valid_first_peaks(vals1, proc_sig1, proc_sig2) # Here we need to provide good signal valleys as reference
        sig_max_slopes1, sig_max_slopes2 =  get_valid_first_peaks(vals1, np.diff(proc_sig1), np.diff(proc_sig2))
        
        # =====================  PTT with different fiducial points ================= ###

        ptt_first_peaks = np.mean(np.array(sig_peaks1) - np.array(sig_peaks2))

        ptt_max_slopes = np.mean(np.array(sig_max_slopes1) - np.array(sig_max_slopes2))

        ptt_valleys = np.mean(vals1 - vals2)
        
        col_ptt_mean_first_peaks.append(round(ptt_first_peaks,3))
        wb.update_cell(row,col,round(ptt_first_peaks,3))
        
        col_ptt_mean_max_slopes.append(round(ptt_max_slopes,3))
        wb.update_cell(row,col + bw_length ,round(ptt_max_slopes,3))
        
        col_ptt_mean_valleys.append(round(ptt_valleys,3))
        wb.update_cell(row,col + bw_length*2,round(ptt_valleys,3))
        
        col += 1
        
    row += 1
    col = row
        
    row_ptt_mean_first_peaks.append(col_ptt_mean_first_peaks)
    row_ptt_mean_max_slopes.append(col_ptt_mean_max_slopes)
    row_ptt_mean_valleys.append(col_ptt_mean_valleys)
    
wb.update_cell(row, col-1, 0);
wb.update_cell(row,col + bw_length-1, 0);
wb.update_cell(row,(col + bw_length*2)-1, 0);

print("values are written :)")

In [38]:
# sh.share('ramu@paretotree.com', perm_type='user', role='writer')
fig, axs = plt.subplots(2, figsize = (10, 4))
axs[0].plot(proc_sig1, label = "Blue")
axs[0].plot(sig_peaks1, proc_sig1[sig_peaks1], 'x')
axs[0].plot(vals1, proc_sig1[vals1], 'o')
axs[0].plot(proc_sig2, label = "Green")
axs[0].plot(sig_peaks2, proc_sig2[sig_peaks2], 'x')
axs[0].plot(vals2, proc_sig2[vals2], 'o')
axs[0].legend()

# axs[1].plot((sig_b - min(sig_b))/(max(sig_b) - min(sig_b)) , label = "Blue")
# axs[1].plot((sig_g - min(sig_g))/(max(sig_g) - min(sig_g)), label = "Green")
axs[1].plot(proc_sig1, label = "Signal 1")
axs[1].plot(proc_sig2, label = "Signal 2")
axs[1].legend()
# axs[1].plot(valleys_B, bpass_sigB[valleys_B], 'o')

# plt.plot(valleys_G, bpass_sigG[valleys_G], 'o')
# plt.plot(sig_peaksG, bpass_sigG[sig_peaksG], 'x')
# plt.legend()
plt.show()